In [1]:
from torch_geometric.datasets import QM9


dataset = QM9(root='data/QM9')
print("Number of molecules:", len(dataset))

/usr/local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Extracting data/QM9/raw/qm9.zip
Processing...
100%|██████████| 133885/133885 [04:26<00:00, 501.97it/s]
Done!


Number of molecules: 130831


In [12]:
from functions_to_abstract_data import extract_qm9_data
data_df = extract_qm9_data(dataset, n_samples=2)
print(data_df)


   index             smiles  num_atoms       HOMO      LUMO        gap  \
0      0  [H]C([H])([H])[H]          5 -10.549854  3.186453  13.736308   
1      1       [H]N([H])[H]          4  -6.993326  2.255824   9.249149   

                                               atoms  \
0  [tensor(6), tensor(1), tensor(1), tensor(1), t...   
1       [tensor(7), tensor(1), tensor(1), tensor(1)]   

                                                 pos  \
0  [[tensor(-0.0127), tensor(1.0858), tensor(0.00...   
1  [[tensor(-0.0404), tensor(1.0241), tensor(0.06...   

                                            edge_idx  \
0  [[tensor(0), tensor(0), tensor(0), tensor(0), ...   
1  [[tensor(0), tensor(0), tensor(0), tensor(1), ...   

                                           edge_attr  
0  [[tensor(1.), tensor(0.), tensor(0.), tensor(0...  
1  [[tensor(1.), tensor(0.), tensor(0.), tensor(0...  
